**Применим метод к модели gemma-2b**

In [ ]:
# Вводим в терминал:
! python3 -m venv venv source venv/bin/activate

In [ ]:
! pip install -r requirements.txt
#перезапустить среду

In [ ]:
#клонируем репозиторий shortened-llama
! git clone https://github.com/Nota-NetsPresso/shortened-llm.git
%cd shortened-llm

In [ ]:
# создаем новый файл prune_gemma.sh, если такого нет
! touch script/prune_gemma.sh
! chmod +x script/prune_gemma.sh

In [ ]:
#добавляем туда виртуальное окружение
! source ../venv/bin/activate

In [ ]:
#обрезка модели Gemma с использованием Perplexity
!python prune_model.py \
--model_name_or_path google/gemma-2b \
--output_dir pruned_gemma_2b \
--pruning_criterion ppl \
--rank 128

In [ ]:
#запускаем скрипт
! bash script/prune_gemma.sh

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "pruned_gemma_2b"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


input_text = "Какой-либо пример для генерации" # ???
input_ids = tokenizer.encode(input_text, return_tensors="pt")

output = model.generate(input_ids, max_length=50)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


generated_text


### Дообучение

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# Загрузка обрезанной модели Gemma
model_name = "./pruned_gemma_2b"  # Путь к вашей обрезанной модели
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Загрузка набора данных StackOverflow
dataset = load_dataset('stackoverflow', split='train[:5%]')

# Подготовка данных (токенизация)
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

trainer.train()
